# Building Language Models from Scratch

In this notebook, we explore the statistical foundations of Natural Language Processing. We will build three types of language models to estimate the probability of a sentence $P(w)$:

1.  **Uniform Model**: All tokens are equally likely.
2.  **Unigram Model**: Probabilities are based on word frequency.
3.  **N-Gram Model**: Probabilities depend on the previous $N-1$ tokens (The Markov Assumption).

We will conclude by training an N-Gram model on Shakespeare to generate synthetic Shakespearean text.

In [26]:
import pandas as pd
import numpy as np
import re
import requests
import time
from pathlib import Path

# Set random seed for reproducibility
np.random.seed(42)

## 1. Data Acquisition & Preprocessing

To train a model, we first need data. The function below fetches books from [Project Gutenberg](https://www.gutenberg.org/) and strips out metadata headers/footers. We then `tokenize` the text, splitting it into words and punctuation while preserving paragraph structures using `\x02` (Start) and `\x03` (Stop) tokens.

In [27]:
def get_book(url):
    """
    Fetches a book from Project Gutenberg and strips metadata.
    """
    robots_url = "https://www.gutenberg.org/robots.txt"
    try:
        # Respect robots.txt (simple delay)
        time.sleep(0.5)
    except:
        pass
        
    r = requests.get(url)
    text = r.text.replace('\r\n', '\n')
    
    # Regex to find the actual start and end of the book content
    start_pattern = r"^\*+\s*START\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+EBOOK[^\n]*\*+$"
    end_pattern = r"^\*+\s*END\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+EBOOK[^\n]*\*+$"
    
    start_match = re.search(start_pattern, text, re.IGNORECASE | re.MULTILINE)
    end_match = re.search(end_pattern, text, re.IGNORECASE | re.MULTILINE)
    
    if not start_match or not end_match:
        # Fallback if patterns aren't found (return full text)
        return text
        
    start_idx = start_match.end()
    end_idx = end_match.start()
    return text[start_idx:end_idx]

def tokenize(book_string):
    """
    Tokenizes text into a list of tokens, adding START (\x02) 
    and STOP (\x03) tokens for paragraphs.
    """
    book_string = book_string.strip()
    if not book_string:
        return ['\x02', '\x03']

    paragraphs = re.split(r'\n{2,}', book_string)
    tokens = []

    for para in paragraphs:
        para = para.strip()
        if not para:
            continue
        tokens.append('\x02')
        # Split by words or punctuation
        para_tokens = re.findall(r'\w+|[^\w\s]', para, re.UNICODE)
        tokens.extend(para_tokens)
        tokens.append('\x03')

    return tokens

## 2. Baseline Models

Before building the N-Gram model, we establish two baselines:
* **UniformLM**: $P(w) = \frac{1}{|V|}$ where $V$ is the vocabulary size.
* **UnigramLM**: $P(w) = \frac{Count(w)}{N}$ where $N$ is total tokens.

In [28]:
class UniformLM:
    def __init__(self, tokenized_corpus):
        self.mdl = None
        self.train(tokenized_corpus)

    def train(self, tokens):
        unique_tokens = pd.Series(list(set(tokens)))
        prob = 1 / len(unique_tokens)
        self.mdl = pd.Series(prob, index=unique_tokens)
        return self.mdl

    def probability(self, token_tuple):
        if any(token not in self.mdl.index for token in token_tuple):
            return 0
        probs = self.mdl.loc[list(token_tuple)]
        return np.prod(probs)

    def sample(self, M):
        sampled_tokens = np.random.choice(self.mdl.index, size=M, replace=True)
        return ' '.join(sampled_tokens)

class UnigramLM:
    def __init__(self, tokens):
        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        token_series = pd.Series(tokens)
        counts = token_series.value_counts()
        probs = counts / counts.sum()
        return probs
    
    def probability(self, words):
        if any(word not in self.mdl.index for words in words):
            return 0
        probs = self.mdl.loc[list(words)]
        return np.prod(probs)
        
    def sample(self, M):
        sampled_tokens = np.random.choice(self.mdl.index, size=M, replace=True, p=self.mdl.values)
        return ' '.join(sampled_tokens)

## 3. The N-Gram Model

The N-Gram model assumes that the probability of a token depends only on the previous $N-1$ tokens:

$$P(w_n|w_1,\ldots,w_{n-1}) \approx P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

We estimate these probabilities using empirical counts from our training corpus:

$$P(w_n | w_{n-1}) = \frac{C(w_{n-1}, w_n)}{C(w_{n-1})}$$

The class below implements this logic using pandas DataFrames to store the conditional probabilities. It also includes a recursive structure (`prev_mdl`) to handle the start of sentences where fewer than $N-1$ tokens exist.

In [29]:
class NGramLM:
    def __init__(self, N, tokens):
        self.N = N
        ngrams = self.create_ngrams(tokens)
        self.ngrams = ngrams
        self.mdl = self.train(ngrams)

        if N < 2:
            raise Exception('N must be greater than 1')
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            self.prev_mdl = NGramLM(N-1, tokens)

    def create_ngrams(self, tokens):
        """Creates a list of sliding window N-grams."""
        return [tuple(tokens[i:i+self.N]) for i in range(len(tokens)-self.N+1)]
        
    def train(self, ngrams):
        """Calculates conditional probabilities for the N-grams."""
        counts = {}
        n1_counts = {}
    
        for ng in ngrams:
            counts[ng] = counts.get(ng, 0) + 1
            n1 = ng[:-1]
            n1_counts[n1] = n1_counts.get(n1, 0) + 1
    
        rows = []
        for ng, c in counts.items():
            n1 = ng[:-1]
            prob = c / n1_counts[n1]
            rows.append({'ngram': ng, 'n1gram': n1, 'prob': prob})
    
        return pd.DataFrame(rows)

    def probability(self, words):
        """Calculates the probability of a sequence of words."""
        words = tuple(words)
        if len(words) == 0:
            return 0.0
    
        prob = 1.0
        for i in range(len(words)):
            context_len = min(i, self.N - 1)
            context = words[i - context_len:i]
            target = words[i]
            ngram = context + (target,)
    
            if len(ngram) < self.N:
                model = self.prev_mdl
                while hasattr(model, 'N') and model.N > len(ngram):
                     model = model.prev_mdl
                
                if isinstance(model, UnigramLM):
                     if target not in model.mdl.index: return 0.0
                     prob *= float(model.mdl[target])
                else:
                    prob_df = model.mdl.set_index('ngram')['prob']
                    if ngram not in prob_df.index: return 0.0
                    prob *= float(prob_df[ngram])
            else:
                prob_df = self.mdl.set_index('ngram')['prob']
                if ngram not in prob_df.index:
                    return 0.0
                prob *= float(prob_df[ngram])
        return prob

    def sample(self, M):
        """Generates a sentence of length M using backoff for the start."""
        output = ['\x02']
        
        while len(output) < M + 1:
            current_model = self
            

            while hasattr(current_model, 'N') and len(output) < current_model.N:
                 current_model = current_model.prev_mdl
            
            if isinstance(current_model, UnigramLM):
                next_token = np.random.choice(current_model.mdl.index, p=current_model.mdl.values)
                output.append(next_token)
                continue

            context = tuple(output[-(current_model.N-1):])
            candidates = current_model.mdl[current_model.mdl['n1gram'] == context]
            
            if candidates.empty:
                output.append('\x03')
                break
            
            next_token = np.random.choice(
                candidates['ngram'].apply(lambda x: x[-1]),
                p=candidates['prob']
            )
            output.append(next_token)
            
        return ' '.join(output[1:])

## 4. Model Demonstration: Shakespeare

Let's test our model on the Complete Works of Shakespeare. We will verify that the **N-Gram model** produces significantly more coherent text than the **Unigram model**.

In [30]:
# 1. Fetch Data
# We download the full text of Shakespeare from Project Gutenberg
url = 'https://www.gutenberg.org/ebooks/100.txt.utf-8' 
print("Downloading corpus...")
shakespeare_text = get_book(url)
tokens = tokenize(shakespeare_text)
print(f"Tokenization complete. Total tokens: {len(tokens)}")

# 2. Train Models
print("Training Unigram Model...")
unigram_model = UnigramLM(tokens)

print("Training Trigram (N=3) Model...")
trigram_model = NGramLM(N=3, tokens=tokens)

# 3. Compare Output
print("\n--- Unigram Sample (Random Words) ---")
print(unigram_model.sample(20))

print("\n--- Trigram Sample (Context-Aware) ---")
print(trigram_model.sample(50))

Tokenization complete. Total tokens: 1340412
Training Unigram Model...
Training Trigram (N=3) Model...

--- Unigram Sample (Random Words) ---
not Sad play see   , PAULINA such Than , disasters Into ’   a A have of

--- Trigram Sample (Context-Aware) ---
“ Your meat doth burn my study , let him live .   SPEED . [ _To Northumberland_ . ] O my liege , Whom I would do were as fleet , That if one be good , That thee may glorify the Lord protect him from me are
